# https://fivethirtyeight.com/features/can-you-find-your-pills/

## Riddler Classic

I’ve been prescribed to take 1.5 pills of a certain medication every day for 10 days, so I have a bottle with 15 pills. Each morning, I take two pills out of the bottle at random.

On the first morning, these are guaranteed to be two full pills. I consume one of them, split the other in half using a precision blade, consume half of that second pill, and place the remaining half back into the bottle.

On subsequent mornings when I take out two pills, there are three possibilities:

* I get two full pills. As on the first morning, I split one and place the unused half back into the bottle.
* I get one full pill and one half-pill, both of which I consume.
* I get two half-pills. In this case, I take out another pill at random. If it’s a half-pill, then I consume all three halves. But if it’s a full pill, I split it and place the unused half back in the bottle.

Assume that each pill — whether it is a full pill or a half-pill — is equally likely to be taken out of the bottle.

On the 10th day, I again take out two pills and consume them. In a rush, I immediately throw the bottle in the trash before bothering to check whether I had just consumed full pills or half-pills. What’s the probability that I took the full dosage, meaning I don’t have to dig through the trash for a remaining half-pill?

### Solution

On Day 1, I will pick 2 full pills and split one, leaving 13 full pills and 1 half pill.

On Day 2, I could pick 2 full pills, with $p = \frac{13}{14} \cdot \frac{12}{13} = \frac{6}{7}$, leaving 11 full pills and 2 half pills, or I could pick 1 full pill and 1 half pill with $p = \frac{13}{14} \cdot \frac{1}{13} + \frac{1}{14} \cdot \frac{13}{13} = \frac{1}{7}$, leaving 12 full pills.

Consider a day with $F$ full pills, $H$ hall pills, and $T = F + H$ total pills. Then the possible scenarios are summarized below:

| Pills picked | Probability | Pills left |
| :---: | :---: | :--- |
| 2 full | $$\begin{cases} 0  & F \leq 1 \\ \frac{F(F-1)}{T(T-1)} & F \geq 2 \end{cases}$$ | F - 2 full <br /> H + 1 half <br /> T - 1 total |
|1 full, 1 half | $$\begin{cases} 0  & F = 0 \text{ or } H = 0 \\ \frac{2FH}{T(T-1)} & \text{otherwise} \end{cases}$$ | F - 1 full <br /> H - 1 half <br /> T - 2 total |
|2 half then 1 full | $$\begin{cases} 0  & F = 0 \text{ or } H \leq 1 \\ \frac{H(H-1)F}{T(T-1)(T-2)} & \text{otherwise} \end{cases}$$ | F - 1 full <br /> H - 1 half <br /> T - 2 total |
|2 half then 1 half | $$\begin{cases} 0  & H \leq 2 \\ \frac{H(H-1)(H-2)}{T(T-1)(T-2)} & \text{otherwise} \end{cases}$$ | F full <br /> H - 3 half <br /> T - 3 total |

Let's write some functions to calculate these probabilities, since I think I may have to just brute force this problem.

In [14]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt

In [15]:
def twoFull(F, H):
    if F <= 1:
        return 0
    else:
        return F*(F-1)/((F+H)*(F+H-1))
    
def oneFull_oneHalf(F, H):
    if F == 0 or H == 0:
        return 0
    else:
        return 2*F*H/((F+H)*(F+H-1))

    
def twoHalf_oneFull(F, H):
    if F == 0 or H <= 1:
        return 0
    else:
        return H*(H-1)*F/((F+H)*(F+H-1)*(F+H-2))
    
def twoHalf_oneHalf(F, H):
    if H <= 2:
        return 0
    else:
        return H*(H-1)*(H-2)/((F+H)*(F+H-1)*(F+H-2))
    
def pickPills(F, H):
    T = F + H
    if twoFull(F, H) == 0:
        print(f"Probability of choosing 2 full pills is 0")
    else:
        print(f"Probability of choosing 2 full pills, "
              f"leaving {F-2} full and {H+1} half pills is {round(twoFull(F, H),2)}")
    if oneFull_oneHalf(F, H) == 0:
        print(f"Probability of choosing 1 full pill and 1 half pill is 0")
    else:
        print(f"Probability of choosing 1 full pill and 1 half pill, "
              f"leaving {F-1} full and {H-1} half pills is {round(oneFull_oneHalf(F, H),2)}")        
    if twoHalf_oneFull(F, H) == 0:
        print(f"Probability of choosing 2 half pills then 1 full pill is 0")
    else:
        print(f"Probability of choosing 2 half pills then 1 full pill, "
              f"leaving {F-1} full and {H-1} half pills is {round(twoHalf_oneFull(F, H),2)}")        
    if twoHalf_oneHalf(F, H) == 0:
        print(f"Probability of choosing 2 half pills then 1 half pill is 0")
    else:
        print(f"Probability of choosing 2 half pills then 1 half pill, "
              f"leaving {F} full and {H-3} half pills is {round(twoHalf_oneHalf(F, H),2)}")

We can see the possibilities of each day. Repeating days 1 and 2:

In [16]:
pickPills(15, 0)

Probability of choosing 2 full pills, leaving 13 full and 1 half pills is 1.0
Probability of choosing 1 full pill and 1 half pill is 0
Probability of choosing 2 half pills then 1 full pill is 0
Probability of choosing 2 half pills then 1 half pill is 0


In [17]:
pickPills(13, 1)

Probability of choosing 2 full pills, leaving 11 full and 2 half pills is 0.86
Probability of choosing 1 full pill and 1 half pill, leaving 12 full and 0 half pills is 0.14
Probability of choosing 2 half pills then 1 full pill is 0
Probability of choosing 2 half pills then 1 half pill is 0


Now, I just need a better way of cumulatively keeping track of all the possibilities and probabilities over the 10 days. As a first step, I will construct a dictionary where the keys are tuples of all possible $(F,H)$ combinations. The values will be dictionaries themselves, with keys representing the possible $(F',H')$ combinations reachable from $(F,H)$ and the values will be the probability of getting there. This will hopefully be a map that can then be used to work from Day 1 to Day 10.

In [22]:
pillDic = {}
for F in range(0,16):
    for H in range(0,8):
        pillDic[(F,H)] = {}
        if twoFull(F, H) != 0:
            pillDic[(F,H)][(F-2,H+1)] = twoFull(F, H)
        if oneFull_oneHalf(F, H) != 0:
            pillDic[(F,H)][(F-1,H-1)] = oneFull_oneHalf(F, H)
        if twoHalf_oneFull(F, H) != 0:
            if (F-1,H-1) in pillDic[(F,H)].keys():
                pillDic[(F,H)][(F-1,H-1)] += twoHalf_oneFull(F, H)
            else:
                pillDic[(F,H)][(F-1,H-1)] = twoHalf_oneFull(F, H)
        if twoHalf_oneHalf(F, H) != 0:
            pillDic[(F,H)][(F,H-3)] = twoHalf_oneHalf(F, H)

In [23]:
slimmedDic = {k: v for k, v in pillDic.items() if len(v)>0}
slimmedDic

{(0, 3): {(0, 0): 1.0},
 (0, 4): {(0, 1): 1.0},
 (0, 5): {(0, 2): 1.0},
 (0, 6): {(0, 3): 1.0},
 (0, 7): {(0, 4): 1.0},
 (1, 1): {(0, 0): 1.0},
 (1, 2): {(0, 1): 1.0},
 (1, 3): {(0, 2): 0.75, (1, 0): 0.25},
 (1, 4): {(0, 3): 0.6000000000000001, (1, 1): 0.4},
 (1, 5): {(0, 4): 0.5, (1, 2): 0.5},
 (1, 6): {(0, 5): 0.42857142857142855, (1, 3): 0.5714285714285714},
 (1, 7): {(0, 6): 0.375, (1, 4): 0.625},
 (2, 0): {(0, 1): 1.0},
 (2, 1): {(0, 2): 0.3333333333333333, (1, 0): 0.6666666666666666},
 (2, 2): {(0, 3): 0.16666666666666666, (1, 1): 0.8333333333333333},
 (2, 3): {(0, 4): 0.1, (1, 2): 0.8, (2, 0): 0.1},
 (2, 4): {(0, 5): 0.06666666666666667,
  (1, 3): 0.7333333333333334,
  (2, 1): 0.2},
 (2, 5): {(0, 6): 0.047619047619047616,
  (1, 4): 0.6666666666666666,
  (2, 2): 0.2857142857142857},
 (2, 6): {(0, 7): 0.03571428571428571,
  (1, 5): 0.6071428571428571,
  (2, 3): 0.35714285714285715},
 (2, 7): {(0, 8): 0.027777777777777776,
  (1, 6): 0.5555555555555556,
  (2, 4): 0.4166666666666667}

Now that I have the dictionary which, given a configuration of $(F,H)$ full, half pills, returns the possible scenarios after choosing pills along with the probability of reaching those states. Now I can just go through all possibilities starting with $(15, 0)$ on Day 1 to see what happens on Day 10.

In [32]:
def addDay(possibleConfigs):
    day = len(possibleConfigs) + 1
    newConfigs = {}
    for config, prob in possibleConfigs[day-1].items():
        for newConfig, newProb in slimmedDic[config].items():
            if newConfig in newConfigs.keys():
                newConfigs[newConfig] += newProb*prob
            else:
                newConfigs[newConfig] = newProb*prob
    possibleConfigs[day] = newConfigs

In [33]:
possibleConfigs = {1: {(15,0): 1.0}}
addDay(possibleConfigs)
print(possibleConfigs)

{1: {(15, 0): 1.0}, 2: {(13, 1): 1.0}}


In [34]:
addDay(possibleConfigs)
print(possibleConfigs)

{1: {(15, 0): 1.0}, 2: {(13, 1): 1.0}, 3: {(11, 2): 0.8571428571428571, (12, 0): 0.14285714285714285}}


Things look good for the first two days, so I will proceed to Day 10.

In [37]:
possibleConfigs = {1: {(15,0): 1.0}}
while len(possibleConfigs) < 10:
    addDay(possibleConfigs)
possibleConfigs

{1: {(15, 0): 1.0},
 2: {(13, 1): 1.0},
 3: {(11, 2): 0.8571428571428571, (12, 0): 0.14285714285714285},
 4: {(9, 3): 0.6043956043956044, (10, 1): 0.3956043956043956},
 5: {(7, 4): 0.3296703296703296,
  (8, 2): 0.5956543456543457,
  (9, 0): 0.07467532467532467},
 6: {(5, 5): 0.12587412587412586,
  (6, 3): 0.5664335664335665,
  (7, 1): 0.3076923076923077},
 7: {(3, 6): 0.02797202797202797,
  (4, 4): 0.32342657342657344,
  (5, 2): 0.5649350649350648,
  (6, 0): 0.08366633366633366},
 8: {(1, 7): 0.0023310023310023306,
  (2, 5): 0.08828671328671328,
  (3, 3): 0.5066956852671137,
  (4, 1): 0.40268659911517046},
 9: {(0, 6): 0.005078255078255078,
  (1, 4): 0.16165382236810805,
  (2, 2): 0.6468584986442127,
  (3, 0): 0.1864094239094239},
 10: {(0, 3): 0.20988029827315535, (1, 1): 0.7901197017268444}}

Looks like on Day 10 there is $~21\%$ chance that there will be 3 half pills and a $~79\%$ chance that there is 1 full pill and 1 half pill, meaning there is a $79\%$ chance I don't have to dig through the trash after taking 2 pills and disposing of the bottle!

I'll do one quick check just to make sure at the very least the probabilities each day sum to 1 so nothing obviously went very wrong during these steps.

In [38]:
for day, configDic in possibleConfigs.items():
    totalProb = 0
    for config, prob in configDic.items():
        totalProb += prob
    print(day," : ",totalProb)

1  :  1.0
2  :  1.0
3  :  1.0
4  :  1.0
5  :  1.0
6  :  1.0
7  :  1.0
8  :  0.9999999999999998
9  :  0.9999999999999998
10  :  0.9999999999999998


Close enough!